In [5]:
import socket
import time
import numpy as np
from PIL import Image
import hmac



# Divide the image into into 920 bit chunks
def divide_image_into_pixel_groups(image_path, group_pixel_count=30):
    # Step 1: Loading the Image
    with Image.open(image_path).convert('RGB') as img:
        width, height = img.size
        

        # Step 2: Calculating Group Dimensions
        # Find a suitable width and height for the groups that would approximately contain 1024 pixels
        group_width = int(np.sqrt(group_pixel_count))
        group_height = group_pixel_count // group_width

        # Adjust group_height in case group_width * group_height < 1024
        if group_width * group_height < group_pixel_count:
            group_height += 1

        groups = []

        # Step 3: Dividing the Image
        for y in range(0, height, group_height):
            for x in range(0, width, group_width):
                # Define the bounding box for each group
                box = (x, y, x + group_width, y + group_height)
                # Crop the image to the box to get the group
                group = img.crop(box)
                groups.append(group)

        return groups
    
def add_MAC(groups):
    for i in range(len(groups)):
        mac = hmac.new(b'key', groups[i].tobytes(), 'sha384').digest()
        groups[i] = groups[i].tobytes() + mac
    return groups


def send_bmp_image(file_path, ip, port, p):
    groups = divide_image_into_pixel_groups(file_path)
    groups = add_MAC(groups)

    # Create a UDP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    i =0
    try:
        for pixel_group in groups:
  
            
            # Send chunk index and chunk data
            if np.random.rand() < p:
                sock.sendto(i.to_bytes(4, 'big') + pixel_group , (ip, port))
                time.sleep(0.0001)
            else:
                print("modify, sequence number = ", i)
                mac = hmac.new(b'key', b'modified', 'sha384').digest()
                if np.random.rand() < .5:
                    sock.sendto(i.to_bytes(4, 'big') + b'b' + mac, (ip, port))
                time.sleep(0.0001)
            if i % 10000 == 0:
                print("Sent chunk ", i, " of ", len(groups))
            i += 1
            

        # Send a final message indicating completion
        sock.sendto(b'END', (ip, port))
    finally:
        print("Image sent")
        sock.close()
    return i

# Example usage
file_path = 'output.png'
destination_ip = '0.0.0.0'
destination_port = 23423


# packet success probability
p = .9999

send_bmp_image(file_path, destination_ip, destination_port, p = p)


Sent chunk  0  of  78012
modify, sequence number =  4789
Sent chunk  10000  of  78012
modify, sequence number =  13044
Sent chunk  20000  of  78012
modify, sequence number =  21694
Sent chunk  30000  of  78012
modify, sequence number =  38578
Sent chunk  40000  of  78012
Sent chunk  50000  of  78012
Sent chunk  60000  of  78012
modify, sequence number =  61044
modify, sequence number =  67357
Sent chunk  70000  of  78012
Image sent


78012

In [6]:
groups = divide_image_into_pixel_groups("output.png", group_pixel_count=30)
# combine the groups into a single image

len(groups)




78012

In [7]:
# groups[0] = Image.fromarray(np.zeros(np.array(groups[1]).shape, dtype=np.uint8))
# groups[0]
np.array(groups[1]).shape

(6, 5, 3)

In [8]:
import hmac

len(hmac.new(b'key', b'message', 'sha384').digest())


48